# ensegment: default program

In [9]:
from ensegment import *

## Documentation


Using the default solution, we get the following result on the dev dataset:

In [10]:
Pw = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pw)
with open("../data/input/dev.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

choose spain
this is a test
who represents
experts exchange
speed of art
unclimatechangebody
we are the people
mentionyourfaves
now playing
the walking dead
follow me
we are the people
mentionyourfaves
check domain
big rock
name cheap
apple domains
honesty hour
being human
follow back
social media
30secondstoearth
current ratesoughttogodown
this is insane
what is my name
is it time
let us go
me too
nowthatcherisdead
advice for young journalists


The abbove results show that the model failed to segments some words like "unclimatechangebody", "mentionyourfaves", etc. Interestingly enough, the words "unclimatechangebody" and "mentionyourfaves" do not exist in the count dictionary file "count_1w.txt".

The problem arises from the fact that the probability of unknown words (unavailable in the "count_1w.txt") is considered 1/N in the code, where N is the total number of the words in the corpus. This can be seen in the "Pdist" class where the default "missingfn" is defined as follows:

In [7]:
# self.missingfn = missingfn or (lambda k, N: 1./N)

This phenomenon could cause the probability of some segmented nouns like "un climate change body" which is p1.p2.p3.p4 becomes less than 1/N (p1, p2, p3, p4 are the probability of "un", "climate", "change", "body").

The more reasonable approach is to consider the probability of the unknown words 0. In other words, we need to define the "missingfn" equal to 0, as follows:

In [11]:
Pw = Pdist(data=datafile("../data/count_1w.txt"), missingfn=(lambda k, N: 0))
segmenter = Segment(Pw)
with open("../data/input/dev.txt") as f:
    for line in f:
        print(" ".join(segmenter.segment(line.strip())))

choose spain
this is a test
who represents
experts exchange
speed of art
un climate change body
we are the people
mention your faves
now playing
the walking dead
follow me
we are the people
mention your faves
check domain
big rock
name cheap
apple domains
honesty hour
being human
follow back
social media
3 0 seconds to earth
current rate sought to go down
this is insane
what is my name
is it time
let us go
me too
now thatcher is dead
advice for young journalists


Now, the segmentations are better and the model is able to capture the correct segments in almost all the dev data.
We used the same approach and reached .98 F-score on the dev dataset.
So, this small change to the default solution will be our final submission for HW0.